In [1]:

from google.colab import files
uploaded = files.upload()


!pip install lightgbm prophet shap statsmodels


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from prophet import Prophet
import shap
import statsmodels.api as sm

#  Load & Clean Data
df = pd.read_csv(list(uploaded.keys())[0])
df.rename(columns={'Order Date': 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date']).reset_index(drop=True)
df['Promotion'] = df['Discount'].apply(lambda x: 1 if x > 0 else 0)

#  Monthly Aggregation + Segmentation
df['YearMonth'] = df['Date'].dt.to_period('M').dt.to_timestamp()
monthly_df = df.groupby(['YearMonth', 'Segment']).agg(
    Subscribers=('Customer ID', pd.Series.nunique),
    ARPU=('Sales', lambda x: x.sum() / df.loc[x.index, 'Customer ID'].nunique()),
    Promo_Flag=('Promotion', 'max'),
    Discount_Avg=('Discount', 'mean')
).reset_index().rename(columns={'YearMonth': 'Date'})

monthly_df['MRR'] = monthly_df['Subscribers'] * monthly_df['ARPU']
monthly_df['Log_MRR'] = np.log1p(monthly_df['MRR'])
monthly_df = monthly_df.dropna().reset_index(drop=True)

#  Feature Engineering
for lag in [1, 2, 3]:
    monthly_df[f'sub_lag_{lag}'] = monthly_df.groupby('Segment')['Subscribers'].shift(lag)
    monthly_df[f'arpu_lag_{lag}'] = monthly_df.groupby('Segment')['ARPU'].shift(lag)
    monthly_df[f'discount_lag_{lag}'] = monthly_df.groupby('Segment')['Discount_Avg'].shift(lag)

monthly_df['month'] = monthly_df['Date'].dt.month
monthly_df['quarter'] = monthly_df['Date'].dt.quarter
monthly_df['year'] = monthly_df['Date'].dt.year

monthly_df = monthly_df.dropna().reset_index(drop=True)

#  Segmented ARPU Modeling
segment_dfs = []
segment_models = {}

for segment in monthly_df['Segment'].unique():
    seg_data = monthly_df[monthly_df['Segment'] == segment].copy()
    features = [col for col in seg_data.columns if col.startswith(('arpu_lag_', 'discount_lag_'))] + ['month', 'quarter', 'year', 'Promo_Flag']
    X = seg_data[features]
    y = seg_data['ARPU']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.3)
    model = LGBMRegressor(n_estimators=500, learning_rate=0.01)
    model.fit(X_train, y_train)
    seg_data = seg_data.reset_index(drop=True)
    seg_data['ARPU_Pred'] = model.predict(X)
    segment_dfs.append(seg_data)
    segment_models[segment] = model

segmented_df = pd.concat(segment_dfs).reset_index(drop=True)

# Global Subscriber Modeling
features = [col for col in segmented_df.columns if col.startswith('sub_lag_')] + ['month', 'quarter', 'year', 'Promo_Flag']
X_sub = segmented_df[features]
y_sub = segmented_df['Subscribers']
X_sub_train, X_sub_test, y_sub_train, y_sub_test = train_test_split(X_sub, y_sub, shuffle=False, test_size=0.3)
sub_model = LGBMRegressor(n_estimators=500, learning_rate=0.01)
sub_model.fit(X_sub_train, y_sub_train)
sub_pred = sub_model.predict(X_sub_test)

#  Prophet Log MRR Forecast on Aligned Test Dates
total_mrr_df = segmented_df.groupby('Date').agg({'Log_MRR': 'mean'}).reset_index()
total_mrr_df = total_mrr_df.rename(columns={'Date': 'ds', 'Log_MRR': 'y'})
prophet_model = Prophet(weekly_seasonality=True)
prophet_model.add_seasonality(name='quarterly', period=91.25, fourier_order=8)
prophet_model.fit(total_mrr_df)
test_dates = segmented_df.loc[X_sub_test.index, 'Date'].values
future = pd.DataFrame({'ds': test_dates})
prophet_forecast = prophet_model.predict(future)
prophet_log_yhat = np.expm1(prophet_forecast[['yhat']].values.flatten())

# Reconstruct Hybrid MRR
arpu_aligned = segmented_df.loc[X_sub_test.index, 'ARPU_Pred'].values
mrr_pred_lgbm = sub_pred * arpu_aligned
mrr_actual = segmented_df.loc[X_sub_test.index, 'Subscribers'].values * segmented_df.loc[X_sub_test.index, 'ARPU'].values
hybrid_mrr = (0.6 * mrr_pred_lgbm + 0.4 * prophet_log_yhat)
hybrid_mrr_smoothed = pd.Series(hybrid_mrr).rolling(window=3, min_periods=1, center=True).mean().values

#  SARIMAX Residual Correction
residuals = mrr_actual - hybrid_mrr_smoothed
sarimax_model = sm.tsa.SARIMAX(residuals, order=(1, 0, 0), seasonal_order=(1, 0, 0, 12)).fit(disp=False)
sarimax_pred = sarimax_model.predict(start=0, end=len(residuals)-1)
hybrid_mrr_corrected = hybrid_mrr_smoothed + sarimax_pred

#  Evaluation

def smape(y_true, y_pred):
    mask = (y_true + y_pred) != 0
    return 100 * np.mean(2 * np.abs(y_pred[mask] - y_true[mask]) / (np.abs(y_pred[mask]) + np.abs(y_true[mask])))

def weighted_smape(y_true, y_pred):
    weight = y_true / np.sum(y_true)
    smape_val = 2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))
    return 100 * np.sum(weight * smape_val)

valid_mask = mrr_actual > 10000
mae = mean_absolute_error(mrr_actual[valid_mask], hybrid_mrr_corrected[valid_mask])
rmse = np.sqrt(mean_squared_error(mrr_actual[valid_mask], hybrid_mrr_corrected[valid_mask]))
smape_score = smape(mrr_actual[valid_mask], hybrid_mrr_corrected[valid_mask])
wsmape_score = weighted_smape(mrr_actual[valid_mask], hybrid_mrr_corrected[valid_mask])

print(f"MAE: ${mae:,.2f}")
print(f"RMSE: ${rmse:,.2f}")
print(f"SMAPE (Filtered >$10k): {smape_score:.2f}%")
print(f"Weighted SMAPE (Filtered): {wsmape_score:.2f}%")

#  Export Forecast
forecast_df = pd.DataFrame({
    'Date': segmented_df.loc[X_sub_test.index, 'Date'].values,
    'Segment': segmented_df.loc[X_sub_test.index, 'Segment'].values,
    'Predicted_Subscribers': sub_pred.round(),
    'Predicted_ARPU': np.round(arpu_aligned, 2),
    'Hybrid_Predicted_MRR': np.round(hybrid_mrr_corrected, 2),
    'Actual_MRR': np.round(mrr_actual, 2)
})

from google.colab import files
forecast_df.to_csv("Best_MRR_Forecast_Final.csv", index=False)
files.download("Best_MRR_Forecast_Final.csv")


Saving SaaS-Sales.csv to SaaS-Sales.csv
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 31, number of used features: 0
[LightGBM] [Info] Start training from score 453.404060
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because t

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvlh2j78f/xqbscuhn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvlh2j78f/xylnenzh.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=88032', 'data', 'file=/tmp/tmpvlh2j78f/xqbscuhn.json', 'init=/tmp/tmpvlh2j78f/xylnenzh.json', 'output', 'file=/tmp/tmpvlh2j78f/prophet_modelssuve61p/prophet_model-20250619184406.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
18:44:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:44:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


MAE: $4,517.13
RMSE: $6,169.17
SMAPE (Filtered >$10k): 23.13%
Weighted SMAPE (Filtered): 26.19%


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>